---

## 0. 환경 설정하기

### 1) 구글 드라이브 연결하기

In [1]:
# 코랩 사용 시 구글 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 2) 경로 확인하기
- "WORK_SPACE" 에 본인 작업 경로 작성 후 실행(구글 드라이브 최상위에 압축해제 시 그대로 실행. 수정 X).<br>

<font color="red">※ 주의. 나머지 경로는 절대 변경하지 마세요.</font>

In [2]:
# ROOT_PATH 확인 
import os

# 구글 드라이브 내 프로젝트 압축해제된 영역 (구글 드라이브 최상위에 압축해제 시 그대로 수정 X)
WORK_SPACE = "kt aivle/mini3"

if os.getcwd() == '/content' :
  # 구글 드라이브 사용 시 
  ROOT_PATH = "/content/drive/MyDrive/"+WORK_SPACE+"/AIVLE3rd_individual"
else :
  ROOT_PATH = os.path.abspath('..')

# 영상을 분할한 이미지 경로
IMAGE_PATH = ROOT_PATH + "/image"
# Train 데이터 셋 경로
TRAIN_PATH = ROOT_PATH + "/train"


### 3) 라이브러리 불러오기
필요시 추가 라이브러리는 설치해서 사용하세요.

In [3]:
# 필요 라이브러리 불러오기.
import os
import glob
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator

---

## 1. 학습 데이터 세트 만들기 

<font color="green">[실습문제]</font> 1. 이미지 데이터를 class 별로 학습을 위한 데이터 세트 만들기
* class명을 출연진으로 하여 class 별로 학습 데이터 세트를 만듭니다.<br>(class명 : 'jung', 'park', 'sam', 'sayuri')
* 학습 데이터 경로 TRIAN_PATH : "../train/"

> [예시] 폴더 구조 <br>
> train ┐ <br>
> &emsp;&emsp; ├ jung <br>
> &emsp;&emsp; ├ park <br>
> &emsp;&emsp; ├ sam <br>
> &emsp;&emsp; ├ sayuri <br>

__※ 방법은 자유롭게 하셔도 됩니다 물리적으로 직접 "copy & paste"를 하셔도 되고 여력이 된다면 코드로 작성하여 자동화를 하셔도 됩니다.__

In [ ]:
# 실습해보세요.






## 2. 데이터 전처리
+ KeyPoint : 케라스에서는 이미지 파일을 쉽게 학습시킬 수 있도록 ImageDataGenerator 클래스를 제공합니다.


### 1) ImageDataGenerator 
+ ImageDataGenerator 클래스를 이용하여 객체를 생성한 뒤 flow_from_directory()함수를 호출하여 Generator를 생성
> + 첫 번째 인자 : 이미지 경로를 지정합니다.
> + target_size : 패치 이미지 크기를 지정합니다. 폴더에 있는 원본 이미지 크기가 다르더라도 target_size에 지정된 크기로 자동 조절됩니다.
> + batch_size : 배치 크기를 지정합니다.
> + class_mode : 분류 방식에 대해서 지정합니다.
>> + categorical : 2D one-hot 부호화된 라벨이 반환됩니다.
>> + binary : 1D 이진 라벨이 반환됩니다.
>> + sparse : 1D 정수 라벨이 반환됩니다.
>> + None : 라벨이 반환되지 않습니다. 

[참고] https://keras.io/ko/preprocessing/image/


<font color="green">[실습문제]</font> 2. ImageDataGenerator 생성하기
+ 모델 검증을 위해 데이터를 train:validation(8:2)로 분할합니다.
+ Data augmentation이 필요 시 자유롭게 설정

In [4]:
# 실습해보세요.
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1. /255,
    validation_split=0.2,
    #rotation_range=40,
    #width_shift_range=0.2,
    #height_shift_range=0.2,
    #shear_range=0.2,
    #zoom_range=0.2,
    #horizontal_flip=True,
    #fill_mode='nearest,
    
)

In [5]:
batch_size=16
img_height = 480
img_width=854

In [6]:
# train_genrator 생성
train_generator = train_datagen.flow_from_directory(
    TRAIN_PATH,
    batch_size=16,
    target_size=(img_height, img_width),
    class_mode='categorical',
    subset='training',

)

Found 695 images belonging to 4 classes.


In [7]:
# validation_generator 생성
validation_generator = train_datagen.flow_from_directory(
    TRAIN_PATH,
    batch_size=16,
    target_size=(img_height, img_width),
    subset='validation',
    class_mode='categorical',



    
)

Found 172 images belonging to 4 classes.


In [ ]:
# 실습해보세요.
import matplotlib.pyplot as plt
%matplotlib inline

class_map = {
    0: 'jung',
    1: 'park',    
    2: 'sam',
    3: 'sayuri',
}

image_class = glob.glob(IMAGE_PATH + "*/*")
image_class.sort()

fig, axes = plt.subplots(4, 5,figsize=(16, 9))
class_count = 0 
for c in image_class:  
  for i in range(5):
    original_image = cv2.imread(c + "/" + str(i*200) + ".jpg")
    rgb_image = cv2.cvtColor(original_image,cv2.COLOR_BGR2RGB)
    # 특정 영역만 잘라서 이미지 비교
    cropped_image = rgb_image[0:96, 0:320]
    axes[class_count, i].imshow(cropped_image)
    axes[class_count, i].set_title(class_map[class_count] , fontsize=15)    
    axes[class_count, i].axis('off')
  class_count += 1
plt.show()

In [ ]:
# [Tip] 
# 이미지 전체를 학습할 필요가 없다. 
# 코너를 구분하는데 필요한 영역은 좌측 상단에 고정되어 있음으로 해당 영역만 잘라내서 학습 데이터를 생성하는 것도 가능

def croppedImage(IMAGE_PATH):
  #이미지를 잘라낼 영역 
  bounding_box = {
      'start_x' : 0, 
      'start_y' : 0,
      'end_x' : 320,    
      'end_y' : 96
  }
  from tqdm.auto import tqdm, trange
  image_list = glob.glob(IMAGE_PATH + '/*/*.jpg')
  for image in tqdm(image_list, desc="IMAGE CROP PROGRESS"):
    original_image = cv2.imread(image)
    cropped_image = original_image[bounding_box['start_y']:bounding_box['end_y'], bounding_box['start_x']:bounding_box['end_x']]    
    cv2.imwrite(image, cropped_image) 